# Setup

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

from deap import algorithms, base, creator, tools

In [19]:
# Toy data
toy_data = pd.DataFrame(data=[[(0.011, 5),(0.03, 6),(0.05, 7),(0.067, 8),(0.09, 9)],
            [(1, 9),(0.67, 6),(0.33, 3),(0.11, 1),(0.05, 5)],
            [(0.75, 0.5),(2.25, 3),(3.5, 5),(5.3, 8),(6.5, 10)],
            [(5.3, 0.1),(9.1, 0.8),(5.04, 0.05),(6.4, 0.3),(8.02, 0.6)]])

In [42]:
MA_df = pd.DataFrame(columns=['activator','inhibitor'],data=[[0,-2],[0.5,-0.7],[0.7,-0.3],[2,0]])
MA_df

,activator,inhibitor
0,0.0,-2.0
1,0.5,-0.7
2,0.7,-0.3
3,2.0,0.0


# DEAP

In [3]:
creator.create(name = 'fitness',
               base = base.Fitness,
               weights = (-1.0,))

creator.create(name = 'individual',
               base = np.ndarray,
               fitness = creator.fitness)

toolbox = base.Toolbox()

/opt/miniconda3/envs/sym-seq-ml/lib/python3.10/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'fitness' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/opt/miniconda3/envs/sym-seq-ml/lib/python3.10/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [167]:
random.seed(42)

def create_individual(individual_class: type, data: pd.DataFrame):
    dimensions = data.shape
    locations = random.sample(range(dimensions[1]),dimensions[0])
    individual = individual_class(data.iloc[i,locations[i]] for i in data.index)

    return individual

In [168]:
toolbox.register(alias = 'individual',
                 function = create_individual,
                 individual_class = creator.individual,
                 data = toy_data) # CHANGE

toolbox.register('population',
                 tools.initRepeat,
                 list,
                 toolbox.individual)

In [236]:
test_population = toolbox.population(n=3)
test_population

[individual([[0.09, 9.  ],
             [0.33, 3.  ],
             [0.75, 0.5 ],
             [9.1 , 0.8 ]]),
 individual([[0.05, 7.  ],
             [0.11, 1.  ],
             [0.75, 0.5 ],
             [8.02, 0.6 ]]),
 individual([[0.011, 5.   ],
             [0.67 , 6.   ],
             [3.5  , 5.   ],
             [8.02 , 0.6  ]])]

In [169]:
# Evaluate
def objective(individual: object, MA_df: pd.DataFrame):
    dot_product = []
    for i in MA_df.index:
        dot_product.append((np.dot([MA_df.iloc[i,0],MA_df.iloc[i,1]],individual[i]))**2)
    sum_dots = np.sum(dot_product)
    
    return -sum_dots,

toolbox.register(alias = 'evaluate', function = objective, MA_df = MA_df)

In [242]:
toolbox.evaluate(test_population[0])

(-301.49125000000004,)

In [170]:
# NOTE: Not sure what appropriate tournament size should be
toolbox.register("select", tools.selTournament, tournsize=5)

# NOTE: Using np.ndarray individuals requires a different method to compare individuals
hof = tools.HallOfFame(3, similar=np.array_equal)

In [208]:
# Mutation
def mutate(individual: object, prob: float, data: pd.DataFrame):
    dimensions = data.shape
    for i, vals in enumerate(individual):
        if random.random() < prob:
            individual[i] = list(data.iloc[i,random.sample(range(dimensions[1]),1)])[0]

    return individual,

toolbox.register('mutate', function = mutate, prob = 1, data = toy_data) # CHANGE

In [241]:
test_iteration = toolbox.mutate(test_population[0])
test_iteration

(individual([[0.05, 7.  ],
             [0.33, 3.  ],
             [0.75, 0.5 ],
             [5.04, 0.05]]),)

In [218]:
# Crossover
toolbox.register('mate', tools.cxESBlend, alpha=0.1)

In [219]:
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

In [231]:
MU = 10
LAMBDA = 20
pop = toolbox.population(n=30)

pop, logbook = algorithms.eaMuCommaLambda(pop, toolbox, mu=MU, lambda_=LAMBDA, 
            cxpb=0, mutpb=0.3, ngen=1000, stats=stats, halloffame=hof, verbose=False)

In [232]:
logbook

[{'gen': 0,
  'nevals': 30,
  'avg': -391.878265,
  'std': 118.14486402435051,
  'min': -690.5961,
  'max': -218.260725},
 {'gen': 1,
  'nevals': 8,
  'avg': -560.9216225,
  'std': 48.0810128191075,
  'min': -601.7181249999999,
  'max': -453.014325},
 {'gen': 2,
  'nevals': 6,
  'avg': -584.0209824999998,
  'std': 27.829955541293632,
  'min': -601.7181249999999,
  'max': -529.1223249999999},
 {'gen': 3,
  'nevals': 6,
  'avg': -600.4647299999999,
  'std': 9.372018709987215,
  'min': -616.6377,
  'max': -591.1248499999999},
 {'gen': 4,
  'nevals': 6,
  'avg': -624.1068724999999,
  'std': 34.203472845071694,
  'min': -691.2824999999999,
  'max': -601.7181249999999},
 {'gen': 5,
  'nevals': 4,
  'avg': -658.440665,
  'std': 40.49862418735851,
  'min': -691.2824999999999,
  'max': -601.7181249999999},
 {'gen': 6,
  'nevals': 4,
  'avg': -691.2825,
  'std': 1.1368683772161603e-13,
  'min': -691.2824999999999,
  'max': -691.2824999999999},
 {'gen': 7,
  'nevals': 7,
  'avg': -691.2825,
  'st

In [233]:
toolbox.evaluate(hof[0])

(-691.2824999999999,)

In [243]:
hof[0]

individual([[ 0.09,  9.  ],
            [ 1.  ,  9.  ],
            [ 6.5 , 10.  ],
            [ 9.1 ,  0.8 ]])

In [244]:
MA_df

,activator,inhibitor
0,0.0,-2.0
1,0.5,-0.7
2,0.7,-0.3
3,2.0,0.0


In [245]:
np.dot(MA_df.iloc[0,:],hof[0][0])

-18.0

In [ ]:
vs code